In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sub = pd.read_csv("/kaggle/input/datathon-2024/sample_submission.csv")
test = pd.read_csv("/kaggle/input/datathon-2024/test_x.csv")
train = pd.read_csv("/kaggle/input/datathon-2024/train.csv")

/tmp/ipykernel_17/1788601659.py:3: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/kaggle/input/datathon-2024/train.csv")


## Veri keşfi

In [3]:
train

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,...,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,...,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,...,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,...,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65123


In [4]:
# şuan korelasyona bakamıyoruz çünkü string falan filan
#train.corr()

In [5]:
print(train["Basvuru Yili"].nunique())
train["Basvuru Yili"].unique()

9


array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022])

In [6]:
# Kategorik, sayısal, metinsel veri

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu

## Veri temizliği:
> #### NaN değerlerle ilgilen
> #### Cinsiyet kısmında Erkek ve ERKEK şeklinde iki değer var.
> #### Doğum tarihlerinden saati çıkar + 25 Mayis şeklinde de yazılıyor.
> #### Doğum yeri kısmında ilçe yazanlar var... Düzeltilsin. Aynı şekilde ikametgah adresinde de.
> #### Üniversite türü - Devlet DEVLET var yine..
> #### Burs yüzdesinde NaN değer vardı.
> #### Burs yüzdesi olup, burs alıyor mu "hayır" seçeneği, sanırım ek olarak anlamında.
> #### Bölüm'de "Truzim ve otel" yazıyor mesela. Ayrıca "Diğer" ne demek adam kendi bölümüne "diğer" mi yazmış?
> #### Üniversite kaçıncı sınıf -> Hazırlık - hazırlık
> #### Not ortalaması kategorik.
> #### Daha önce mezun olmuş mu sütunu "Null"
> #### Lise adı, bilmiyorum.. Null değeri var ama garipti.
> #### lise adı diğer null ve yok çok var.
> #### Lise şehir isimlendirmeleri ANKARA ankara düzeltilsin.
> #### Lise türü "diğer" diğer ne demek abi? Devlet ve özel yazmış lise türüne.
> #### lise bölümü EA eşit ağırlık Eşit Ağırlık...
> #### lise bölüm diğer %98 null.
> #### lise mezuniyet notu 3.00 da var, 75 de var.
> #### başka bir kurumdan burs alıyor mu? sorusu var.. Eee?
> #### Burs aldığı başka kurum: Kyk kyK kyk
> #### Başka kurumdan aldığı burs miktarı 500 ve üzeri, 500₺..
> #### Anne eğitim durumu ilkokul ilk okul EĞİTİM YOK
> #### anne çalışma durumu null
> #### anne sektör -> 0
> #### baba eğitim
> #### baba çalışma null
> #### baba sektör tekrar eden..
> #### kardeş sayısı 1.0 XD 18...?
> #### g.klüp null
> #### klüp ismi %97 null
> #### spor dalıyla meşgul müsün, null %2
> #### spor dalındaki rolün 0, null..
> #### stk üyesi %22 diğer
> #### hangi stk -> - ve null
> #### stk projesi null
> #### girişimcilik deneyimi evet hayır sorusunda diğer oranı.
> #### girişim. deneyim. açıkla -> A B Ü -
> #### İng. biliyon mu %75 evet.
> #### İng. seviye 0 , %56 null.
> #### tekrarlayan değer. mezun. üni.
> #### id sorunsuz.

# Genel Düzenleme -- Değiştirildi
Toplam train verisi 65125 satır, 44 sütun
> #### Cinsiyet
- 115 Belirmek istemiyorum ve 169 null kaldırıldı.
- 10012 ERKEK -> Erkek olarak değiştirildi.

### Cinsiyet

In [7]:
print(train["Cinsiyet"].nunique())
train["Cinsiyet"].unique()

4


array(['Erkek', 'Kadın', 'ERKEK', 'Belirtmek istemiyorum', nan],
      dtype=object)

In [8]:
train["Cinsiyet"].isna().sum()

169

In [9]:
len(train.loc[train["Cinsiyet"] == "Belirtmek istemiyorum"])

115

In [10]:
train['Cinsiyet'] = train['Cinsiyet'].replace('ERKEK', 'Erkek')

In [11]:
train = train.dropna(subset=["Cinsiyet"])

In [12]:
train = train.loc[train['Cinsiyet'] != 'Belirtmek istemiyorum']

### Dogum Tarihi

In [13]:
print(train["Dogum Tarihi"].nunique())
train["Dogum Tarihi"].unique()

13488


array(['4/6/1994', '6/11/1993', '1/15/1986', ..., '10.04.2003 00:00',
       '13.12.1997 00:00', '12.11.2003 00:00'], dtype=object)

In [14]:
train['Dogum Tarihi'] = train['Dogum Tarihi'].str.extract(r'(\d{1,2}[/\.]\d{1,2}[/\.]\d{2,4})')

In [15]:
print(train["Dogum Tarihi"].nunique())
train["Dogum Tarihi"].unique()

9239


array(['4/6/1994', '6/11/1993', '1/15/1986', ..., '10.04.2003',
       '13.12.1997', '12.11.2003'], dtype=object)

In [16]:
train.loc[train["Dogum Tarihi"] == "1204"]

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,...,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id


# Aşağıyı bi silme, Aralik Mayis yazan çok var. Onları /ile birleştirttireyim. 14471 veri boş çıkıyordu önceki kodda

In [17]:
train["Dogum Tarihi"].isna().sum()

14357

In [18]:
#train.loc[train.index >= 52212]

# Hızlı ilerleyiş submission için 10.09.2024
- #### Veriler kodlanacak kategorize neydi ismi herneyse
- #### Toplam train verisi 65125 satır, 44 sütun
- #### Alınacak veriler (26 tane, gün bitmeden yollayabilmek için, belki azaltırım zaman yetmezse):
> ##### id
> ##### Başvuru yılı
> ##### değerlendirme p.
> ##### Cinsiyet
> ##### Universite Turu
> ##### Burslu ise burs yüzdesi
> ##### Burs Alayi mi
> ##### uni kac sinif
> ##### uni not ort.
> ##### onceden baska uni?
> ##### lise mezuniyet notu
> ##### baska kurumdan burs?
> ##### anne eğitim durumu
> ##### anne çalışma durumu
> ##### anne sektor
> ##### baba eğitim
> ##### baba çalışma
> ##### baba sektör
> ##### kardes sayisi
> ##### bir klübe üye misin?
> ##### spor dalıyla meşgul mü?
> ##### stk üyeliği?
> ##### stk projesine katildiniz mi?
> ##### girişimcilik deneyi?
> ##### ing. biliyon mu?
> ##### ing. seviyen?

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [20]:
sub = pd.read_csv("/kaggle/input/datathon-2024/sample_submission.csv")
test = pd.read_csv("/kaggle/input/datathon-2024/test_x.csv")
train = pd.read_csv("/kaggle/input/datathon-2024/train.csv")

/tmp/ipykernel_17/1788601659.py:3: DtypeWarning: Columns (13,15,19,30,32,37,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/kaggle/input/datathon-2024/train.csv")


In [21]:
# SÜTUN ADLARINI DÜZELTME
train.columns = [col.strip().lower().replace(' ', '_').replace('?', '').replace("'", '').replace("ö", 'o').replace("ü", 'u').replace("Ö", 'o').replace("Ü", 'u').replace(",", '') for col in train.columns]

In [22]:
train.columns

Index(['basvuru_yili', 'degerlendirme_puani', 'cinsiyet', 'dogum_tarihi',
       'dogum_yeri', 'ikametgah_sehri', 'universite_adi', 'universite_turu',
       'burslu_ise_burs_yuzdesi', 'burs_aliyor_mu', 'bolum',
       'universite_kacinci_sinif', 'universite_not_ortalamasi',
       'daha_once_baska_bir_universiteden_mezun_olmus', 'lise_adi',
       'lise_adi_diger', 'lise_sehir', 'lise_turu', 'lise_bolumu',
       'lise_bolum_diger', 'lise_mezuniyet_notu',
       'baska_bir_kurumdan_burs_aliyor_mu', 'burs_aldigi_baska_kurum',
       'baska_kurumdan_aldigi_burs_miktari', 'anne_egitim_durumu',
       'anne_calisma_durumu', 'anne_sektor', 'baba_egitim_durumu',
       'baba_calisma_durumu', 'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz',
       'uye_oldugunuz_kulubun_ismi',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz',
       'spor_dalindaki_rolunuz_nedir', 'aktif_olarak_bir_stk_uyesi_misiniz',
       'hangi_stknin_uyesisiniz', 'stk_pro

In [23]:
train = train.applymap(lambda x: x.lower() if isinstance(x, str) else x)

/tmp/ipykernel_17/195616094.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train = train.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [24]:
train.isnull().sum()

basvuru_yili                                                    0
degerlendirme_puani                                             1
cinsiyet                                                      169
dogum_tarihi                                                  177
dogum_yeri                                                    791
ikametgah_sehri                                              2037
universite_adi                                                132
universite_turu                                               255
burslu_ise_burs_yuzdesi                                     42685
burs_aliyor_mu                                                  0
bolum                                                         231
universite_kacinci_sinif                                      374
universite_not_ortalamasi                                    2753
daha_once_baska_bir_universiteden_mezun_olmus               37345
lise_adi                                                      384
lise_adi_d

### degerlendirme_puani ve basvuru_yili

In [25]:
print(train["degerlendirme_puani"].isna().sum())
train["basvuru_yili"].isna().sum()

1


0

In [26]:
# NaN satırı görelim:
train[train['degerlendirme_puani'].isna()]

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
65124,2022,NaN,kadın,27.04.2000 00:00,hatay,hatay,gaziantep üniversitesi,devlet,NaN,hayır,...,takım oyuncusu,evet,genç kızılay topluluğu'nda 3 yıldır gönüllüyüm...,NaN,evet,aile desteğiyle yemek sektöründe küçük bir işl...,evet,NaN,NaN,65124


In [27]:
train = train.dropna(subset=["degerlendirme_puani"])
print("degerlendirme_puani NaN veri sayısı ",train["degerlendirme_puani"].isna().sum())

degerlendirme_puani NaN veri sayısı  0


### cinsiyet
- 115 Belirmek istemiyorum ve 169 null kaldırıldı.

In [28]:
print(train["cinsiyet"].nunique())
train["cinsiyet"].unique()

3


array(['erkek', 'kadın', 'belirtmek istemiyorum', nan], dtype=object)

In [29]:
train["cinsiyet"].isna().sum()

169

In [30]:
len(train.loc[train["cinsiyet"] == "belirtmek istemiyorum"])

115

In [31]:
train = train.dropna(subset=["cinsiyet"])

In [32]:
train = train.loc[train['cinsiyet'] != 'belirtmek istemiyorum']

In [33]:
# GÜNCEL DURUM
print("cinsiyet kategorisi yeni değer sayısı:", train["cinsiyet"].nunique())
print("cinsiyet kategorisi yeni değerler:",train["cinsiyet"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

cinsiyet kategorisi yeni değer sayısı: 2
cinsiyet kategorisi yeni değerler: ['erkek' 'kadın']
Eğitim seti yeni satır sayısı 64840


### universite_turu
- 221 null kaldırıldı.

In [34]:
print(train["universite_turu"].nunique())
train["universite_turu"].unique()

2


array(['özel', 'devlet', nan], dtype=object)

In [35]:
train["universite_turu"].isna().sum()

221

In [36]:
train[train["universite_turu"].isna()].head()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
3432,2020,57.0,kadın,1/1/70 2:00,i̇stanbul,i̇stanbul,i̇stanbul okan üni̇versi̇tesi̇,NaN,100.0,evet,...,NaN,evet,aysit vakfı,NaN,evet,kanada -toronto’da bulunduğum süre içerisinde ...,evet,NaN,NaN,3432
3578,2020,30.0,kadın,1/1/70 2:00,trabzon,ankara,beykent üni̇versi̇tesi̇,NaN,100.0,evet,...,NaN,evet,kaçuv,NaN,hayır,NaN,evet,NaN,NaN,3578
3944,2020,47.0,erkek,4/3/97 0:00,i̇stanbul,yalova,doğuş üni̇versi̇tesi̇,NaN,100.0,evet,...,lider/kaptan,hayır,NaN,NaN,evet,"meetzon, doğru mekanları kullanıcılarıyla bulu...",evet,NaN,NaN,3944
3987,2020,43.0,kadın,1/1/70 2:00,i̇stanbul,gaziantep,beykent üni̇versi̇tesi̇,NaN,100.0,evet,...,NaN,hayır,NaN,NaN,evet,i̇lk başta kullanmadığım kıyafetleri gardrops ...,hayır,NaN,NaN,3987
4053,2021,32.0,kadın,11/4/02 0:00,i̇stanbul,yalova,koç üni̇versi̇tesi̇,NaN,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,4053


In [37]:
train = train.dropna(subset=["universite_turu"])

In [38]:
# GÜNCEL DURUM
print("universite_turu kategorisi yeni değer sayısı:", train["universite_turu"].nunique())
print("universite_turu kategorisi yeni değerler:",train["universite_turu"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

universite_turu kategorisi yeni değer sayısı: 2
universite_turu kategorisi yeni değerler: ['özel' 'devlet']
Eğitim seti yeni satır sayısı 64619


### Burslu ise Burs Yuzdesi

- 42269 NaN veri, 0 olarak değiştirildi.

In [39]:
print(train["burslu_ise_burs_yuzdesi"].nunique())
train["burslu_ise_burs_yuzdesi"].unique()

50


array([100. ,  99. ,  98. ,  91.5,  90. ,  88. ,  87.5,  85. ,  82.5,
        80. ,  75. ,  74. ,  72. ,  70. ,  69. ,  67. ,  65. ,  64. ,
        63. ,  62.5,  62. ,  60. ,  57.5,  57. ,  55. ,  50. ,  49. ,
        48. ,  46. ,  45. ,  41.5,  41. ,  40. ,  37. ,  35. ,  32. ,
        30. ,  25. ,  20. ,  11. ,  10. ,   8. ,   7. ,   6. ,   5. ,
         4. ,   3. ,   2. ,   1. ,   0. ,   nan])

In [40]:
train["burslu_ise_burs_yuzdesi"].isna().sum()

42268

In [41]:
train["burslu_ise_burs_yuzdesi"] = train["burslu_ise_burs_yuzdesi"].fillna(0)

In [42]:
# GÜNCEL DURUM
print("burslu_ise_burs_yuzdesi kategorisi yeni değer sayısı:", train["burslu_ise_burs_yuzdesi"].nunique())
print("burslu_ise_burs_yuzdesi kategorisi yeni değerler:",train["burslu_ise_burs_yuzdesi"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

burslu_ise_burs_yuzdesi kategorisi yeni değer sayısı: 50
burslu_ise_burs_yuzdesi kategorisi yeni değerler: [100.   99.   98.   91.5  90.   88.   87.5  85.   82.5  80.   75.   74.
  72.   70.   69.   67.   65.   64.   63.   62.5  62.   60.   57.5  57.
  55.   50.   49.   48.   46.   45.   41.5  41.   40.   37.   35.   32.
  30.   25.   20.   11.   10.    8.    7.    6.    5.    4.    3.    2.
   1.    0. ]
Eğitim seti yeni satır sayısı 64619


### burs_aliyor_mu

- Boş veri; ilginçtir ki, hiç yoktu.

In [43]:
print(train["burs_aliyor_mu"].nunique())
train["burs_aliyor_mu"].unique()

2


array(['evet', 'hayır'], dtype=object)

In [44]:
train["burs_aliyor_mu"].isna().sum()

0

In [45]:
# GÜNCEL DURUM
print("burs_aliyor_mu kategorisi yeni değer sayısı:", train["burs_aliyor_mu"].nunique())
print("burs_aliyor_mu kategorisi yeni değerler:",train["burs_aliyor_mu"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

burs_aliyor_mu kategorisi yeni değer sayısı: 2
burs_aliyor_mu kategorisi yeni değerler: ['evet' 'hayır']
Eğitim seti yeni satır sayısı 64619


### universite_kacinci_sinif ! ! !

- 0 değerine sahip olanlar kaldırıldı. (3 kayıt vardı.) 

In [46]:
print(train["universite_kacinci_sinif"].nunique())
train["universite_kacinci_sinif"].unique()

11


array(['3', '1', nan, '2', '4', 'hazırlık', 'mezun', 'yüksek lisans', '5',
       '6', 'tez', '0'], dtype=object)

In [47]:
train.loc[train["universite_kacinci_sinif"] == "0"]

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
12818,2014,10.0,erkek,8/15/1993,kahramanmaraş,NaN,gazi̇antep üni̇versi̇tesi̇,devlet,0.0,evet,...,0,hayır,NaN,hayır,hayır,NaN,NaN,NaN,NaN,12818
14605,2014,14.0,erkek,1/18/1994,balıkesir,i̇stanbul,boğazi̇çi̇ üni̇versi̇tesi̇,devlet,0.0,hayır,...,0,hayır,NaN,hayır,evet,"3 şirket deneyimim var1 seo ajansı, 1 eticaret...",NaN,NaN,NaN,14605
15589,2014,44.0,kadın,1/1/1970,yerköy,kırşehir,orta doğu tekni̇k üni̇versi̇tesi̇,devlet,0.0,hayır,...,diğer,evet,"sulak alanları koruma, yaban hayatını koruma, ...",hayır,evet,içinde benim üye olduğum sivil toplum kuruluşu...,NaN,NaN,NaN,15589


In [48]:
print(len(train.loc[train["universite_kacinci_sinif"] == "0"]))
print(len(train.loc[train["universite_kacinci_sinif"] == "tez"]))
print(len(train.loc[train["universite_kacinci_sinif"] == "yüksek lisans"])) # yeni kategori açmak için iyi bir sayı değil. Sistemi yormaya değmez.
print(len(train.loc[train["universite_kacinci_sinif"] == "mezun"]))
print(len(train.loc[train["universite_kacinci_sinif"] == "hazırlık"]))

3
7
15
178
1289


In [49]:
train = train.loc[train['universite_kacinci_sinif'] != '0']
train = train.loc[train['universite_kacinci_sinif'] != 'tez']
train = train.loc[train['universite_kacinci_sinif'] != 'yüksek lisans']

In [50]:
train = train.dropna(subset=["universite_kacinci_sinif"])

In [51]:
train["universite_kacinci_sinif"].isna().sum()

0

In [52]:
# GÜNCEL DURUM
print("universite_kacinci_sinif kategorisi yeni değer sayısı:", train["universite_kacinci_sinif"].nunique())
print("universite_kacinci_sinif kategorisi yeni değerler:",train["universite_kacinci_sinif"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

universite_kacinci_sinif kategorisi yeni değer sayısı: 8
universite_kacinci_sinif kategorisi yeni değerler: ['3' '1' '2' '4' 'hazırlık' 'mezun' '5' '6']
Eğitim seti yeni satır sayısı 64276


### universite_not_ortalamasi ! ! !

-

In [53]:
print(train["universite_not_ortalamasi"].nunique())
train["universite_not_ortalamasi"].unique()

19


array(['3.50-3', '3.00-2.50', nan, '2.50 ve altı', '3.00 - 4.00',
       '3.50 - 4.00', '3.00 - 3.50', '2.50 -3.00', 'ortalama bulunmuyor',
       '4-3.5', '2.50 - 3.00', '2.00 - 2.50', '1.00 - 2.50',
       'not ortalaması yok', '4.0-3.5', '3.00 - 3.49', '2.50 - 2.99',
       '1.80 - 2.49', 'hazırlığım', '0 - 1.79'], dtype=object)

In [54]:
train["universite_not_ortalamasi"].value_counts()

universite_not_ortalamasi
3.00-2.50              10675
2.50 ve altı           10511
3.00 - 3.50             7737
3.50-3                  6135
2.50 - 3.00             5621
3.50 - 4.00             4367
ortalama bulunmuyor     2798
2.50 - 2.99             2445
2.00 - 2.50             2439
3.00 - 3.49             2376
2.50 -3.00              1734
3.00 - 4.00             1397
1.80 - 2.49             1166
hazırlığım               667
not ortalaması yok       575
4-3.5                    475
1.00 - 2.50              320
0 - 1.79                 185
4.0-3.5                    1
Name: count, dtype: int64

In [55]:
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('3.00-2.50', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('3.00 - 3.50', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('3.50-3', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('2.50 - 3.00', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('3.50 - 4.00', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('2.50 - 2.99', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('2.00 - 2.50', '2.50 ve altı')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('3.00 - 3.49', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('2.50 -3.00', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('3.00 - 4.00', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('1.80 - 2.49', '2.50 ve altı')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('4-3.5', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('1.00 - 2.50', '2.50 ve altı')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('0 - 1.79', '2.50 ve altı')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('4.0-3.5', '2.50 ve üstü')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('not ortalaması yok', '0')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('ortalama bulunmuyor', '0')
train['universite_not_ortalamasi'] = train['universite_not_ortalamasi'].replace('hazırlığım', '0')

In [56]:
train["universite_not_ortalamasi"] = train["universite_not_ortalamasi"].fillna("0")

In [57]:
train["universite_not_ortalamasi"].value_counts()

universite_not_ortalamasi
2.50 ve üstü    42963
2.50 ve altı    14621
0                6692
Name: count, dtype: int64

In [58]:
train["universite_not_ortalamasi"].isna().sum()

0

In [59]:
train[train['universite_not_ortalamasi'].isna()].head()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id


In [60]:
# GÜNCEL DURUM
print("universite_not_ortalamasi kategorisi yeni değer sayısı:", train["universite_not_ortalamasi"].nunique())
print("universite_not_ortalamasi kategorisi yeni değerler:",train["universite_not_ortalamasi"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

universite_not_ortalamasi kategorisi yeni değer sayısı: 3
universite_not_ortalamasi kategorisi yeni değerler: ['2.50 ve üstü' '0' '2.50 ve altı']
Eğitim seti yeni satır sayısı 64276


### daha_once_baska_bir_universiteden_mezun_olmus

- hayır yerine NaN'lar vardı doldurdum. (Öyle olduğunu varsayıyoruz %90 olasılıkla)

In [61]:
print(train["daha_once_baska_bir_universiteden_mezun_olmus"].nunique())
train["daha_once_baska_bir_universiteden_mezun_olmus"].unique()

2


array(['hayır', 'evet', nan], dtype=object)

In [62]:
train["daha_once_baska_bir_universiteden_mezun_olmus"].isna().sum()

36712

In [63]:
train[train['daha_once_baska_bir_universiteden_mezun_olmus'].isna()].head()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
1757,2017,7.0,kadın,9/20/1998,erzurum,erzurum,kafkas üni̇versi̇tesi̇,devlet,100.0,evet,...,NaN,hayır,NaN,hayır,hayır,NaN,evet,başlangıç,NaN,1757
1758,2017,58.0,erkek,12/26/1995,istanbul,istanbul,sabanci üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,evet,NaN,evet,evet,NaN,evet,i̇leri,NaN,1758
1759,2017,64.0,erkek,8/3/1995,diger,istanbul,i̇stanbul sabahatti̇n zai̇m,özel,100.0,evet,...,kaptan / li̇der,hayır,NaN,evet,evet,NaN,evet,i̇leri,NaN,1759
1760,2017,4.0,erkek,1/1/1997,mardin,istanbul,doğuş üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,hayır,NaN,NaN,1760
1761,2017,38.0,erkek,11/11/1995,sanliurfa,istanbul,i̇stanbul kavram meslek,özel,100.0,evet,...,di̇ğer,hayır,NaN,evet,evet,NaN,evet,başlangıç,NaN,1761


In [64]:
train["daha_once_baska_bir_universiteden_mezun_olmus"] = train["daha_once_baska_bir_universiteden_mezun_olmus"].fillna("hayır")

In [65]:
train["daha_once_baska_bir_universiteden_mezun_olmus"].isna().sum()

0

In [66]:
# GÜNCEL DURUM
print("daha_once_baska_bir_universiteden_mezun_olmus kategorisi yeni değer sayısı:", train["daha_once_baska_bir_universiteden_mezun_olmus"].nunique())
print("daha_once_baska_bir_universiteden_mezun_olmus kategorisi yeni değerler:",train["daha_once_baska_bir_universiteden_mezun_olmus"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

daha_once_baska_bir_universiteden_mezun_olmus kategorisi yeni değer sayısı: 2
daha_once_baska_bir_universiteden_mezun_olmus kategorisi yeni değerler: ['hayır' 'evet']
Eğitim seti yeni satır sayısı 64276


### lise_mezuniyet_notu ! ! !

-

In [67]:
print(train["lise_mezuniyet_notu"].nunique())
train["lise_mezuniyet_notu"].unique()

19


array(['3.50-3', '3.00 - 4.00', '3.00-2.50', '2.50 ve altı', '4.00-3.50',
       '3.50-3.00', nan, '69-55', '100-85', '84-70', '54-45', '44-0',
       '75 - 100', '50 - 75', 'not ortalaması yok', '25 - 50', '0 - 25',
       '50 - 74', '25 - 49', '0 - 24'], dtype=object)

In [68]:
train["lise_mezuniyet_notu"].isna().sum()

5797

In [69]:
train[train['lise_mezuniyet_notu'].isna()].head()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
1210,2016,56.0,erkek,1995-02-12,mersin,mersin,toros üniversitesi,özel,100.0,evet,...,NaN,hayır,NaN,evet,hayır,NaN,evet,i̇leri,NaN,1210
1211,2016,80.0,kadın,1996-09-23,mersin,i̇stanbul,özyeğin üniversitesi,özel,100.0,evet,...,diğer,hayır,NaN,evet,evet,NaN,evet,i̇leri,NaN,1211
1212,2016,64.0,kadın,1996-02-16,manisa,i̇stanbul,yeditepe üniversitesi,özel,100.0,evet,...,diğer,hayır,NaN,hayır,evet,NaN,evet,orta,NaN,1212
1213,2016,84.0,erkek,1996-09-20,erzincan,i̇stanbul,koç üniversitesi,özel,100.0,evet,...,diğer,evet,yeşilay,evet,evet,NaN,evet,i̇leri,NaN,1213
1214,2016,64.0,kadın,1994-09-17,kırklareli,i̇stanbul,i̇stanbul aydın üniversitesi,özel,100.0,evet,...,NaN,hayır,NaN,evet,hayır,NaN,hayır,NaN,NaN,1214


In [70]:
train["lise_mezuniyet_notu"] = train["lise_mezuniyet_notu"].fillna("hayır")

In [71]:
train["lise_mezuniyet_notu"].isna().sum()

0

In [72]:
# GÜNCEL DURUM
print("lise_mezuniyet_notu kategorisi yeni değer sayısı:", train["lise_mezuniyet_notu"].nunique())
print("lise_mezuniyet_notu kategorisi yeni değerler:",train["lise_mezuniyet_notu"].unique())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

lise_mezuniyet_notu kategorisi yeni değer sayısı: 20
lise_mezuniyet_notu kategorisi yeni değerler: ['3.50-3' '3.00 - 4.00' '3.00-2.50' '2.50 ve altı' '4.00-3.50' '3.50-3.00'
 'hayır' '69-55' '100-85' '84-70' '54-45' '44-0' '75 - 100' '50 - 75'
 'not ortalaması yok' '25 - 50' '0 - 25' '50 - 74' '25 - 49' '0 - 24']
Eğitim seti yeni satır sayısı 64276


### baska_bir_kurumdan_burs_aliyor_mu ! ! !

-

In [73]:
print(train["baska_bir_kurumdan_burs_aliyor_mu"].nunique())
train["baska_bir_kurumdan_burs_aliyor_mu"].unique()

2


array(['hayır', 'evet', nan], dtype=object)

In [74]:
train["baska_bir_kurumdan_burs_aliyor_mu"].isna().sum()

204

In [75]:
train[train['baska_bir_kurumdan_burs_aliyor_mu'].isna()].head()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
1760,2017,4.0,erkek,1/1/1997,mardin,istanbul,doğuş üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,hayır,NaN,NaN,1760
1840,2017,6.0,erkek,6/27/1996,istanbul,kocaeli,balikesi̇r üni̇versi̇tesi̇,devlet,100.0,evet,...,NaN,hayır,NaN,hayır,hayır,NaN,evet,orta,NaN,1840
1941,2017,9.0,kadın,5/29/1994,istanbul,istanbul,okan üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,hayır,NaN,NaN,1941
2034,2017,11.0,kadın,7/10/1996,diger,diger,gi̇resun üni̇versi̇tesi̇,devlet,100.0,evet,...,NaN,hayır,NaN,hayır,hayır,NaN,evet,başlangıç,NaN,2034
2054,2017,10.0,kadın,1/1/1994,yozgat,ankara,çankaya üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,hayır,NaN,NaN,2054


In [76]:
train['baska_bir_kurumdan_burs_aliyor_mu'] = train['baska_bir_kurumdan_burs_aliyor_mu'].fillna(train['burs_aliyor_mu'])

In [77]:
train["baska_bir_kurumdan_burs_aliyor_mu"].isna().sum()

0

In [78]:
# GÜNCEL DURUM
print("baska_bir_kurumdan_burs_aliyor_mu kategorisi yeni değer sayısı:", train["baska_bir_kurumdan_burs_aliyor_mu"].nunique())
print("baska_bir_kurumdan_burs_aliyor_mu kategorisi yeni değerler:",train["baska_bir_kurumdan_burs_aliyor_mu"].unique())
print("baska_bir_kurumdan_burs_aliyor_mu null değer sayısı:", train["baska_bir_kurumdan_burs_aliyor_mu"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

baska_bir_kurumdan_burs_aliyor_mu kategorisi yeni değer sayısı: 2
baska_bir_kurumdan_burs_aliyor_mu kategorisi yeni değerler: ['hayır' 'evet']
baska_bir_kurumdan_burs_aliyor_mu null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### anne_egitim_durumu ! ! !

-

In [79]:
print(train["anne_egitim_durumu"].nunique())
train["anne_egitim_durumu"].unique()

18


array(['üniversite', 'i̇lkokul mezunu', 'eğitim yok', 'lise',
       'ortaokul mezunu', 'yüksek lisans / doktora', 'üniversite mezunu',
       'lise mezunu', 'yüksek lisans / doktara', 'li̇se', 'eği̇ti̇m yok',
       nan, 'üni̇versi̇te', 'doktora', 'yüksek li̇sans', 'ortaokul',
       'i̇lkokul', 'yüksek lisans', 'eğitimi yok'], dtype=object)

In [80]:
train["anne_egitim_durumu"].isna().sum()

322

In [81]:
train[train['anne_egitim_durumu'].isna()].tail()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
54737,2021,33.0,kadın,3/24/99 0:00,adana,adana,i̇stanbul üni̇versi̇tesi̇-,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,54737
54950,2021,23.0,erkek,7/30/99 0:00,kayseri,i̇stanbul,NaN,devlet,0.0,hayır,...,lider/kaptan,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,54950
56248,2021,20.0,kadın,4/9/01 0:00,denizli,denizli,pamukkale üni̇versi̇tesi̇,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,56248
56343,2021,5.0,erkek,3/13/01 0:00,diğer,edirne,trakya üni̇versi̇tesi̇,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,hayır,NaN,NaN,56343
56530,2021,14.0,kadın,NaN,NaN,NaN,uşak üni̇versi̇tesi̇,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,56530


In [82]:
train['anne_egitim_durumu'] = train['anne_egitim_durumu'].fillna("eğitim yok")

In [83]:
train["anne_egitim_durumu"].isna().sum()

0

In [84]:
# GÜNCEL DURUM
print("anne_egitim_durumu kategorisi yeni değer sayısı:", train["anne_egitim_durumu"].nunique())
print("anne_egitim_durumu kategorisi yeni değerler:",train["anne_egitim_durumu"].unique())
print("anne_egitim_durumu null değer sayısı:", train["anne_egitim_durumu"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

anne_egitim_durumu kategorisi yeni değer sayısı: 18
anne_egitim_durumu kategorisi yeni değerler: ['üniversite' 'i̇lkokul mezunu' 'eğitim yok' 'lise' 'ortaokul mezunu'
 'yüksek lisans / doktora' 'üniversite mezunu' 'lise mezunu'
 'yüksek lisans / doktara' 'li̇se' 'eği̇ti̇m yok' 'üni̇versi̇te' 'doktora'
 'yüksek li̇sans' 'ortaokul' 'i̇lkokul' 'yüksek lisans' 'eğitimi yok']
anne_egitim_durumu null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### anne_calisma_durumu ! ! !

-

In [85]:
print(train["anne_calisma_durumu"].nunique())
train["anne_calisma_durumu"].unique()

3


array(['hayır', 'evet', 'emekli', nan], dtype=object)

In [86]:
train["anne_calisma_durumu"].isna().sum()

11581

In [87]:
train[train['anne_calisma_durumu'].isna()].tail()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
57498,2021,56.0,kadın,11/13/98 0:00,kocaeli,kocaeli,gebze tekni̇k üni̇versi̇tesi̇,devlet,0.0,hayır,...,lider/kaptan,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57498
57499,2021,5.0,erkek,2/24/99 0:00,sivas,sivas,si̇vas cumhuri̇yet üni̇versi̇tesi̇,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57499
57500,2021,43.0,erkek,3/23/00 0:00,bursa,ankara,i̇hsan doğramaci bi̇lkent üni̇versi̇tesi̇,özel,0.0,hayır,...,diğer,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57500
57501,2021,29.0,kadın,2/1/99 0:00,mardin,mardin,i̇zmi̇r bakirçay üni̇versi̇tesi̇,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57501
57502,2021,66.0,erkek,4/25/00 0:00,ankara,ankara,i̇hsan doğramaci bi̇lkent üni̇versi̇tesi̇,özel,0.0,hayır,...,diğer,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57502


In [88]:
train['anne_calisma_durumu'] = train['anne_calisma_durumu'].fillna("hayır")

In [89]:
train["anne_calisma_durumu"].isna().sum()

0

In [90]:
# GÜNCEL DURUM
print("anne_calisma_durumu kategorisi yeni değer sayısı:", train["anne_calisma_durumu"].nunique())
print("anne_calisma_durumu kategorisi yeni değerler:",train["anne_calisma_durumu"].unique())
print("anne_calisma_durumu null değer sayısı:", train["anne_calisma_durumu"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

anne_calisma_durumu kategorisi yeni değer sayısı: 3
anne_calisma_durumu kategorisi yeni değerler: ['hayır' 'evet' 'emekli']
anne_calisma_durumu null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### anne_sektor ! ! !

-

In [91]:
# 2 tane "diğer" algıladığı için, bunu düzeltme vakti
train["anne_sektor"] = train["anne_sektor"].str.lower()
train["anne_sektor"] = train["anne_sektor"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [92]:
print(train["anne_sektor"].nunique())
train["anne_sektor"].unique()

5


array(['0', 'kamu', 'ozel sektor', 'diger', nan, '-'], dtype=object)

In [93]:
train["anne_sektor"].isna().sum()

33989

In [94]:
train["anne_sektor"].value_counts()

anne_sektor
0              11640
ozel sektor     5967
-               5818
kamu            5094
diger           1768
Name: count, dtype: int64

In [95]:
train['anne_sektor'] = train['anne_sektor'].replace('-', '0')

In [96]:
train[train['anne_sektor'].isna()].tail()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
57497,2021,46.0,kadın,12/10/99 0:00,i̇stanbul,i̇stanbul,gebze tekni̇k üni̇versi̇tesi̇,devlet,0.0,hayır,...,lider/kaptan,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57497
57498,2021,56.0,kadın,11/13/98 0:00,kocaeli,kocaeli,gebze tekni̇k üni̇versi̇tesi̇,devlet,0.0,hayır,...,lider/kaptan,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57498
57499,2021,5.0,erkek,2/24/99 0:00,sivas,sivas,si̇vas cumhuri̇yet üni̇versi̇tesi̇,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57499
57501,2021,29.0,kadın,2/1/99 0:00,mardin,mardin,i̇zmi̇r bakirçay üni̇versi̇tesi̇,devlet,0.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57501
57502,2021,66.0,erkek,4/25/00 0:00,ankara,ankara,i̇hsan doğramaci bi̇lkent üni̇versi̇tesi̇,özel,0.0,hayır,...,diğer,NaN,NaN,NaN,NaN,NaN,evet,NaN,NaN,57502


In [97]:
train['anne_sektor'] = train['anne_sektor'].fillna("0")

In [98]:
train["anne_sektor"].isna().sum()

0

In [99]:
train["anne_sektor"].value_counts()

anne_sektor
0              51447
ozel sektor     5967
kamu            5094
diger           1768
Name: count, dtype: int64

In [100]:
# GÜNCEL DURUM
print("anne_sektor kategorisi yeni değer sayısı:", train["anne_sektor"].nunique())
print("anne_sektor kategorisi yeni değerler:",train["anne_sektor"].unique())
print("anne_sektor null değer sayısı:", train["anne_sektor"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

anne_sektor kategorisi yeni değer sayısı: 4
anne_sektor kategorisi yeni değerler: ['0' 'kamu' 'ozel sektor' 'diger']
anne_sektor null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### baba_egitim_durumu ! ! !

-

In [101]:
train["baba_egitim_durumu"] = train["baba_egitim_durumu"].str.lower()
train["baba_egitim_durumu"] = train["baba_egitim_durumu"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [102]:
print(train["baba_egitim_durumu"].nunique())
train["baba_egitim_durumu"].unique()

15


array(['universite', 'yuksek lisans / doktora', 'ilkokul mezunu', nan,
       'ortaokul mezunu', 'lise', 'egitim yok', '0', 'universite mezunu',
       'lise mezunu', 'yuksek lisans / doktara', 'yuksek lisans',
       'doktora', 'ilkokul', 'ortaokul', 'egitimi yok'], dtype=object)

In [103]:
train["baba_egitim_durumu"].isna().sum()

904

In [104]:
train["baba_egitim_durumu"].value_counts()

baba_egitim_durumu
lise                       14298
universite                 13255
ilkokul mezunu             11094
ilkokul                     8106
ortaokul mezunu             5038
ortaokul                    4026
universite mezunu           1731
lise mezunu                 1405
egitim yok                  1386
egitimi yok                 1085
yuksek lisans                945
doktora                      393
yuksek lisans / doktora      286
yuksek lisans / doktara      215
0                            109
Name: count, dtype: int64

In [105]:
# "yüksek lisans / doktara"ları YÜKSEK LİSANSA YAZIYORUM, FARKETMEZ ŞİMDİLİK

train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('universite mezunu', 'universite')
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('ilkokul mezunu', 'ilkokul')
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('ortaokul mezunu', 'ortaokul')
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('lise mezunu', 'lise')
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('egitim yok', '0')
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('egitimi yok', '0')
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('yuksek lisans / doktora', 'yuksek lisans')
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].replace('yuksek lisans / doktara', 'yuksek lisans')

In [106]:
train[train['baba_egitim_durumu'].isna()].head()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
6,2014,18.0,erkek,10/13/1994,i̇stanbul,i̇stanbul,i̇stanbul medi̇pol üni̇versi̇tesi̇,özel,100.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
25,2014,60.0,kadın,6/9/1993,i̇stanbul,i̇stanbul,i̇stanbul bi̇lgi̇ üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25
52,2014,28.0,kadın,2/17/1993,gross-umstadtalmanya,ankara,i̇hsan doğramaci bi̇lkent,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52
54,2014,34.0,kadın,2/26/1992,sultanbeyli̇,gaziosmanpaşa/i̇stanbul,i̇stanbul 29 mayis üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54
58,2014,38.0,kadın,2/27/1991,gemlik,i̇stanbul,okan üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58


In [107]:
train['baba_egitim_durumu'] = train['baba_egitim_durumu'].fillna("0")

In [108]:
train["baba_egitim_durumu"].value_counts()

baba_egitim_durumu
ilkokul          19200
lise             15703
universite       14986
ortaokul          9064
0                 3484
yuksek lisans     1446
doktora            393
Name: count, dtype: int64

In [109]:
# GÜNCEL DURUM
print("baba_egitim_durumu kategorisi yeni değer sayısı:", train["baba_egitim_durumu"].nunique())
print("baba_egitim_durumu kategorisi yeni değerler:",train["baba_egitim_durumu"].unique())
print("baba_egitim_durumu null değer sayısı:", train["baba_egitim_durumu"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

baba_egitim_durumu kategorisi yeni değer sayısı: 7
baba_egitim_durumu kategorisi yeni değerler: ['universite' 'yuksek lisans' 'ilkokul' '0' 'ortaokul' 'lise' 'doktora']
baba_egitim_durumu null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### baba_calisma_durumu ! ! !

-

In [110]:
print(train["baba_calisma_durumu"].nunique())
train["baba_calisma_durumu"].unique()

3


array(['emekli', 'evet', nan, 'hayır'], dtype=object)

In [111]:
train["baba_calisma_durumu"].isna().sum()

12294

In [112]:
train["baba_calisma_durumu"].value_counts()

baba_calisma_durumu
evet      28284
hayır     12314
emekli    11384
Name: count, dtype: int64

In [113]:
train[train['baba_calisma_durumu'].isna()].head()

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id
6,2014,18.0,erkek,10/13/1994,i̇stanbul,i̇stanbul,i̇stanbul medi̇pol üni̇versi̇tesi̇,özel,100.0,hayır,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
25,2014,60.0,kadın,6/9/1993,i̇stanbul,i̇stanbul,i̇stanbul bi̇lgi̇ üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25
52,2014,28.0,kadın,2/17/1993,gross-umstadtalmanya,ankara,i̇hsan doğramaci bi̇lkent,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52
54,2014,34.0,kadın,2/26/1992,sultanbeyli̇,gaziosmanpaşa/i̇stanbul,i̇stanbul 29 mayis üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,54
58,2014,38.0,kadın,2/27/1991,gemlik,i̇stanbul,okan üni̇versi̇tesi̇,özel,100.0,evet,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58


In [114]:
# NaN verileri babası çalışmıyor'a yazdım. 

train['baba_calisma_durumu'] = train['baba_calisma_durumu'].fillna("hayır")

In [115]:
train["baba_calisma_durumu"].value_counts()

baba_calisma_durumu
evet      28284
hayır     24608
emekli    11384
Name: count, dtype: int64

In [116]:
# GÜNCEL DURUM
print("baba_calisma_durumu kategorisi yeni değer sayısı:", train["baba_calisma_durumu"].nunique())
print("baba_calisma_durumu kategorisi yeni değerler:",train["baba_calisma_durumu"].unique())
print("baba_calisma_durumu null değer sayısı:", train["baba_calisma_durumu"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

baba_calisma_durumu kategorisi yeni değer sayısı: 3
baba_calisma_durumu kategorisi yeni değerler: ['emekli' 'evet' 'hayır']
baba_calisma_durumu null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### baba_sektor ! ! !

-

In [117]:
train["baba_sektor"] = train["baba_sektor"].str.lower()
train["baba_sektor"] = train["baba_sektor"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [118]:
print(train["baba_sektor"].nunique())
train["baba_sektor"].unique()

5


array(['0', 'ozel sektor', 'kamu', nan, 'diger', '-'], dtype=object)

In [119]:
train["baba_sektor"].isna().sum()

20018

In [120]:
train["baba_sektor"].value_counts()

baba_sektor
ozel sektor    15408
kamu           10233
diger           9006
0               6548
-               3063
Name: count, dtype: int64

In [121]:
train['baba_sektor'] = train['baba_sektor'].replace('-', '0')

In [122]:
# NaN verileri babası çalışmıyor'a yazdım. 

train['baba_sektor'] = train['baba_sektor'].fillna("0")

In [123]:
train["baba_sektor"].value_counts()

baba_sektor
0              29629
ozel sektor    15408
kamu           10233
diger           9006
Name: count, dtype: int64

In [124]:
# GÜNCEL DURUM
print("baba_sektor kategorisi yeni değer sayısı:", train["baba_sektor"].nunique())
print("baba_sektor kategorisi yeni değerler:",train["baba_sektor"].unique())
print("baba_sektor null değer sayısı:", train["baba_sektor"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

baba_sektor kategorisi yeni değer sayısı: 4
baba_sektor kategorisi yeni değerler: ['0' 'ozel sektor' 'kamu' 'diger']
baba_sektor null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### kardes_sayisi ! ! !

-

In [125]:
train["kardes_sayisi"] = pd.to_numeric(train["kardes_sayisi"], errors='coerce')

In [126]:
print(train["kardes_sayisi"].nunique())
train["kardes_sayisi"].unique()

15


array([ 1.,  6., 10.,  3., nan,  4.,  2.,  5.,  0.,  7.,  8., 11.,  9.,
       12., 13., 14.])

In [127]:
train["kardes_sayisi"].value_counts()

kardes_sayisi
1.0     21980
2.0     16814
3.0      8716
4.0      5548
0.0      5006
5.0      4090
6.0       339
7.0       272
8.0       171
10.0      168
9.0       122
11.0       11
12.0        5
13.0        2
14.0        1
Name: count, dtype: int64

In [128]:
train.loc[train["kardes_sayisi"] == "kardeş sayısı 1 ek bilgi aile hk. anne vefat"]

,basvuru_yili,degerlendirme_puani,cinsiyet,dogum_tarihi,dogum_yeri,ikametgah_sehri,universite_adi,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,...,spor_dalindaki_rolunuz_nedir,aktif_olarak_bir_stk_uyesi_misiniz,hangi_stknin_uyesisiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,girisimcilikle_ilgili_deneyiminizi_aciklayabilir_misiniz,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,daha_onceden_mezun_olunduysa_mezun_olunan_universite,id


In [129]:
train = train[train['kardes_sayisi'] != 'kardeş sayısı 1 ek bilgi aile hk. anne vefat']

In [130]:
train["kardes_sayisi"].isna().sum()

1031

In [131]:
# NaN verileri babası çalışmıyor'a yazdım. 

train['kardes_sayisi'] = train['kardes_sayisi'].fillna("0")

In [132]:
train["kardes_sayisi"].value_counts()

kardes_sayisi
1.0     21980
2.0     16814
3.0      8716
4.0      5548
0.0      5006
5.0      4090
0        1031
6.0       339
7.0       272
8.0       171
10.0      168
9.0       122
11.0       11
12.0        5
13.0        2
14.0        1
Name: count, dtype: int64

In [133]:
# GÜNCEL DURUM
print("kardes_sayisi kategorisi yeni değer sayısı:", train["kardes_sayisi"].nunique())
print("kardes_sayisi kategorisi yeni değerler:",train["kardes_sayisi"].unique())
print("kardes_sayisi null değer sayısı:", train["kardes_sayisi"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

kardes_sayisi kategorisi yeni değer sayısı: 16
kardes_sayisi kategorisi yeni değerler: [1.0 6.0 10.0 3.0 '0' 4.0 2.0 5.0 0.0 7.0 8.0 11.0 9.0 12.0 13.0 14.0]
kardes_sayisi null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz ! ! !

-

In [134]:
print(train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].nunique())
train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].unique()

2


array(['evet', 'hayır', nan], dtype=object)

In [135]:
train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].value_counts()

girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz
hayır    33265
evet     29341
Name: count, dtype: int64

In [136]:
train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].isna().sum()

1670

In [137]:
# NaN verileri babası çalışmıyor'a yazdım. 

train['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'] = train['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'].fillna("hayır")

In [138]:
train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].value_counts()

girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz
hayır    34935
evet     29341
Name: count, dtype: int64

In [139]:
# GÜNCEL DURUM
print("girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz kategorisi yeni değer sayısı:", train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].nunique())
print("girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz kategorisi yeni değerler:",train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].unique())
print("girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz null değer sayısı:", train["girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz kategorisi yeni değer sayısı: 2
girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz kategorisi yeni değerler: ['evet' 'hayır']
girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### profesyonel_bir_spor_daliyla_mesgul_musunuz ! ! !

-

In [140]:
print(train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].nunique())
train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].unique()

2


array(['evet', 'hayır', nan], dtype=object)

In [141]:
train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].value_counts()

profesyonel_bir_spor_daliyla_mesgul_musunuz
hayır    41904
evet     21305
Name: count, dtype: int64

In [142]:
train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].isna().sum()

1067

In [143]:
# NaN verileri babası çalışmıyor'a yazdım. 

train['profesyonel_bir_spor_daliyla_mesgul_musunuz'] = train['profesyonel_bir_spor_daliyla_mesgul_musunuz'].fillna("hayır")

In [144]:
train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].value_counts()

profesyonel_bir_spor_daliyla_mesgul_musunuz
hayır    42971
evet     21305
Name: count, dtype: int64

In [145]:
# GÜNCEL DURUM
print("profesyonel_bir_spor_daliyla_mesgul_musunuz kategorisi yeni değer sayısı:", train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].nunique())
print("profesyonel_bir_spor_daliyla_mesgul_musunuz kategorisi yeni değerler:",train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].unique())
print("profesyonel_bir_spor_daliyla_mesgul_musunuz null değer sayısı:", train["profesyonel_bir_spor_daliyla_mesgul_musunuz"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

profesyonel_bir_spor_daliyla_mesgul_musunuz kategorisi yeni değer sayısı: 2
profesyonel_bir_spor_daliyla_mesgul_musunuz kategorisi yeni değerler: ['evet' 'hayır']
profesyonel_bir_spor_daliyla_mesgul_musunuz null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### aktif_olarak_bir_stk_uyesi_misiniz ! ! !

-

In [146]:
print(train["aktif_olarak_bir_stk_uyesi_misiniz"].nunique())
train["aktif_olarak_bir_stk_uyesi_misiniz"].unique()

2


array(['hayır', nan, 'evet'], dtype=object)

In [147]:
train["aktif_olarak_bir_stk_uyesi_misiniz"].value_counts()

aktif_olarak_bir_stk_uyesi_misiniz
hayır    37282
evet     14487
Name: count, dtype: int64

In [148]:
train["aktif_olarak_bir_stk_uyesi_misiniz"].isna().sum()

12507

In [149]:
train['aktif_olarak_bir_stk_uyesi_misiniz'] = train['aktif_olarak_bir_stk_uyesi_misiniz'].fillna("hayır")

In [150]:
train["aktif_olarak_bir_stk_uyesi_misiniz"].value_counts()

aktif_olarak_bir_stk_uyesi_misiniz
hayır    49789
evet     14487
Name: count, dtype: int64

In [151]:
# GÜNCEL DURUM
print("aktif_olarak_bir_stk_uyesi_misiniz kategorisi yeni değer sayısı:", train["aktif_olarak_bir_stk_uyesi_misiniz"].nunique())
print("aktif_olarak_bir_stk_uyesi_misiniz kategorisi yeni değerler:",train["aktif_olarak_bir_stk_uyesi_misiniz"].unique())
print("aktif_olarak_bir_stk_uyesi_misiniz null değer sayısı:", train["aktif_olarak_bir_stk_uyesi_misiniz"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

aktif_olarak_bir_stk_uyesi_misiniz kategorisi yeni değer sayısı: 2
aktif_olarak_bir_stk_uyesi_misiniz kategorisi yeni değerler: ['hayır' 'evet']
aktif_olarak_bir_stk_uyesi_misiniz null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### stk_projesine_katildiniz_mi ! ! !

-

In [152]:
print(train["stk_projesine_katildiniz_mi"].nunique())
train["stk_projesine_katildiniz_mi"].unique()

2


array(['evet', 'hayır', nan], dtype=object)

In [153]:
train["stk_projesine_katildiniz_mi"].value_counts()

stk_projesine_katildiniz_mi
hayır    19507
evet     17087
Name: count, dtype: int64

In [154]:
train["stk_projesine_katildiniz_mi"].isna().sum()

27682

In [155]:
train['stk_projesine_katildiniz_mi'] = train['stk_projesine_katildiniz_mi'].fillna("hayır")

In [156]:
train["stk_projesine_katildiniz_mi"].value_counts()

stk_projesine_katildiniz_mi
hayır    47189
evet     17087
Name: count, dtype: int64

In [157]:
# GÜNCEL DURUM
print("stk_projesine_katildiniz_mi kategorisi yeni değer sayısı:", train["stk_projesine_katildiniz_mi"].nunique())
print("stk_projesine_katildiniz_mi kategorisi yeni değerler:",train["stk_projesine_katildiniz_mi"].unique())
print("stk_projesine_katildiniz_mi null değer sayısı:", train["stk_projesine_katildiniz_mi"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

stk_projesine_katildiniz_mi kategorisi yeni değer sayısı: 2
stk_projesine_katildiniz_mi kategorisi yeni değerler: ['evet' 'hayır']
stk_projesine_katildiniz_mi null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### girisimcilikle_ilgili_deneyiminiz_var_mi ! ! !

-

In [158]:
print(train["girisimcilikle_ilgili_deneyiminiz_var_mi"].nunique())
train["girisimcilikle_ilgili_deneyiminiz_var_mi"].unique()

2


array(['hayır', 'evet', nan], dtype=object)

In [159]:
train["girisimcilikle_ilgili_deneyiminiz_var_mi"].value_counts()

girisimcilikle_ilgili_deneyiminiz_var_mi
hayır    33532
evet     18233
Name: count, dtype: int64

In [160]:
train["girisimcilikle_ilgili_deneyiminiz_var_mi"].isna().sum()

12511

In [161]:
train['girisimcilikle_ilgili_deneyiminiz_var_mi'] = train['girisimcilikle_ilgili_deneyiminiz_var_mi'].fillna("hayır")

In [162]:
train["girisimcilikle_ilgili_deneyiminiz_var_mi"].value_counts()

girisimcilikle_ilgili_deneyiminiz_var_mi
hayır    46043
evet     18233
Name: count, dtype: int64

In [163]:
# GÜNCEL DURUM
print("girisimcilikle_ilgili_deneyiminiz_var_mi kategorisi yeni değer sayısı:", train["girisimcilikle_ilgili_deneyiminiz_var_mi"].nunique())
print("girisimcilikle_ilgili_deneyiminiz_var_mi kategorisi yeni değerler:",train["girisimcilikle_ilgili_deneyiminiz_var_mi"].unique())
print("girisimcilikle_ilgili_deneyiminiz_var_mi null değer sayısı:", train["girisimcilikle_ilgili_deneyiminiz_var_mi"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

girisimcilikle_ilgili_deneyiminiz_var_mi kategorisi yeni değer sayısı: 2
girisimcilikle_ilgili_deneyiminiz_var_mi kategorisi yeni değerler: ['hayır' 'evet']
girisimcilikle_ilgili_deneyiminiz_var_mi null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### ingilizce_biliyor_musunuz ! ! !

-

In [164]:
print(train["ingilizce_biliyor_musunuz"].nunique())
train["ingilizce_biliyor_musunuz"].unique()

2


array([nan, 'evet', 'hayır'], dtype=object)

In [165]:
train["ingilizce_biliyor_musunuz"].value_counts()

ingilizce_biliyor_musunuz
evet     48394
hayır     9524
Name: count, dtype: int64

In [166]:
train["ingilizce_biliyor_musunuz"].isna().sum()

6358

In [167]:
train['ingilizce_biliyor_musunuz'] = train['ingilizce_biliyor_musunuz'].fillna("hayır")

In [168]:
train["ingilizce_biliyor_musunuz"].value_counts()

ingilizce_biliyor_musunuz
evet     48394
hayır    15882
Name: count, dtype: int64

In [169]:
# GÜNCEL DURUM
print("ingilizce_biliyor_musunuz kategorisi yeni değer sayısı:", train["ingilizce_biliyor_musunuz"].nunique())
print("ingilizce_biliyor_musunuz kategorisi yeni değerler:",train["ingilizce_biliyor_musunuz"].unique())
print("ingilizce_biliyor_musunuz null değer sayısı:", train["ingilizce_biliyor_musunuz"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

ingilizce_biliyor_musunuz kategorisi yeni değer sayısı: 2
ingilizce_biliyor_musunuz kategorisi yeni değerler: ['hayır' 'evet']
ingilizce_biliyor_musunuz null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


### ingilizce_seviyeniz ! ! !

-

In [170]:
train["ingilizce_seviyeniz"] = train["ingilizce_seviyeniz"].str.lower()
train["ingilizce_seviyeniz"] = train["ingilizce_seviyeniz"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [171]:
print(train["ingilizce_seviyeniz"].nunique())
train["ingilizce_seviyeniz"].unique()

4


array([nan, 'ileri', 'orta', '0', 'baslangc'], dtype=object)

In [172]:
train["ingilizce_seviyeniz"].value_counts()

ingilizce_seviyeniz
orta        12998
ileri        8296
baslangc     5161
0            2010
Name: count, dtype: int64

In [173]:
train["ingilizce_seviyeniz"].isna().sum()

35811

In [174]:
train['ingilizce_seviyeniz'] = train['ingilizce_seviyeniz'].fillna("0")

In [175]:
train["ingilizce_seviyeniz"].value_counts()

ingilizce_seviyeniz
0           37821
orta        12998
ileri        8296
baslangc     5161
Name: count, dtype: int64

In [176]:
# GÜNCEL DURUM
print("ingilizce_seviyeniz kategorisi yeni değer sayısı:", train["ingilizce_seviyeniz"].nunique())
print("ingilizce_seviyeniz kategorisi yeni değerler:",train["ingilizce_seviyeniz"].unique())
print("ingilizce_seviyeniz null değer sayısı:", train["ingilizce_seviyeniz"].isna().sum())

print(f"Eğitim seti yeni satır sayısı {len(train)}")

ingilizce_seviyeniz kategorisi yeni değer sayısı: 4
ingilizce_seviyeniz kategorisi yeni değerler: ['0' 'ileri' 'orta' 'baslangc']
ingilizce_seviyeniz null değer sayısı: 0
Eğitim seti yeni satır sayısı 64276


# Mapping & Feature Selection

In [177]:
# 'selected_columns' listesinde tutmak istediğin sütunlar olacak.
# Normalde olmasına rağmen hızlanmam gerektiği için (submission zamanını kaçırmamak adına), lise_mezuniyet_notu ve anne_egitim_durumu'nu almıyorum.
selected_columns = ['basvuru_yili', 'degerlendirme_puani', 'cinsiyet',
       'universite_turu',
       'burslu_ise_burs_yuzdesi', 'burs_aliyor_mu',
       'universite_kacinci_sinif', 'universite_not_ortalamasi',
       'daha_once_baska_bir_universiteden_mezun_olmus',
       'baska_bir_kurumdan_burs_aliyor_mu',
       'anne_calisma_durumu', 'anne_sektor', 'baba_egitim_durumu',
       'baba_calisma_durumu', 'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz','aktif_olarak_bir_stk_uyesi_misiniz',
       'stk_projesine_katildiniz_mi',
       'girisimcilikle_ilgili_deneyiminiz_var_mi',
       'ingilizce_biliyor_musunuz', 'ingilizce_seviyeniz',
       'id']  # Bu sütunları sakla

train_set = train[selected_columns]  # Geri kalan sütunlar drop edilmiş olur.

In [178]:
train_set

,basvuru_yili,degerlendirme_puani,cinsiyet,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,universite_kacinci_sinif,universite_not_ortalamasi,daha_once_baska_bir_universiteden_mezun_olmus,baska_bir_kurumdan_burs_aliyor_mu,...,baba_sektor,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz,profesyonel_bir_spor_daliyla_mesgul_musunuz,aktif_olarak_bir_stk_uyesi_misiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,id
0,2014,52.0,erkek,özel,100.0,evet,3,2.50 ve üstü,hayır,hayır,...,0,1.0,evet,evet,hayır,evet,hayır,hayır,0,0
1,2014,30.0,erkek,özel,100.0,hayır,3,2.50 ve üstü,hayır,hayır,...,0,1.0,hayır,hayır,hayır,evet,evet,hayır,0,1
2,2014,18.0,erkek,özel,100.0,hayır,1,0,hayır,hayır,...,ozel sektor,6.0,evet,hayır,hayır,hayır,hayır,hayır,0,2
3,2014,40.0,erkek,özel,100.0,evet,3,2.50 ve üstü,hayır,hayır,...,0,10.0,evet,hayır,hayır,evet,hayır,hayır,0,3
5,2014,37.0,erkek,özel,100.0,evet,3,2.50 ve üstü,hayır,hayır,...,kamu,3.0,evet,hayır,hayır,hayır,hayır,hayır,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65119,2022,0.0,erkek,devlet,0.0,hayır,2,2.50 ve altı,hayır,hayır,...,ozel sektor,1.0,hayır,hayır,hayır,hayır,hayır,hayır,0,65119
65120,2022,0.0,erkek,özel,0.0,evet,2,2.50 ve altı,hayır,hayır,...,ozel sektor,4.0,hayır,hayır,hayır,hayır,hayır,hayır,0,65120
65121,2022,0.0,erkek,devlet,0.0,hayır,1,0,hayır,hayır,...,ozel sektor,4.0,hayır,hayır,hayır,hayır,hayır,hayır,0,65121
65122,2022,0.0,erkek,devlet,0.0,hayır,4,2.50 ve altı,hayır,evet,...,0,3.0,hayır,hayır,hayır,hayır,hayır,hayır,0,65122


In [179]:
train_set['cinsiyet'] = train_set['cinsiyet'].map({'erkek': 1, 'kadın': 0})
train_set['universite_turu'] = train_set['universite_turu'].map({'özel': 1, 'devlet': 0})
train_set['burs_aliyor_mu'] = train_set['burs_aliyor_mu'].map({'evet': 1, 'hayır': 0})
train_set['daha_once_baska_bir_universiteden_mezun_olmus'] = train_set['daha_once_baska_bir_universiteden_mezun_olmus'].map({'evet': 1, 'hayır': 0})
train_set['baska_bir_kurumdan_burs_aliyor_mu'] = train_set['baska_bir_kurumdan_burs_aliyor_mu'].map({'evet': 1, 'hayır': 0})
train_set['anne_calisma_durumu'] = train_set['anne_calisma_durumu'].map({'evet': 1, 'hayır': 0, 'emekli':2})
train_set['anne_sektor'] = train_set['anne_sektor'].map({'0': 0, 'kamu': 1, 'ozel sektor':2, 'diger':3})
train_set['baba_calisma_durumu'] = train_set['baba_calisma_durumu'].map({'evet': 1, 'hayır': 0, 'emekli':2})
train_set['baba_sektor'] = train_set['baba_sektor'].map({'0': 0, 'kamu': 1, 'ozel sektor':2, 'diger':3})
train_set['ingilizce_seviyeniz'] = train_set['ingilizce_seviyeniz'].map({'0': 0, 'baslangc': 1, 'orta':2, 'ileri':3})
train_set['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'] = train_set['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'].map({'evet': 1, 'hayır': 0})
train_set['profesyonel_bir_spor_daliyla_mesgul_musunuz'] = train_set['profesyonel_bir_spor_daliyla_mesgul_musunuz'].map({'evet': 1, 'hayır': 0})
train_set['aktif_olarak_bir_stk_uyesi_misiniz'] = train_set['aktif_olarak_bir_stk_uyesi_misiniz'].map({'evet': 1, 'hayır': 0})
train_set['stk_projesine_katildiniz_mi'] = train_set['stk_projesine_katildiniz_mi'].map({'evet': 1, 'hayır': 0})
train_set['girisimcilikle_ilgili_deneyiminiz_var_mi'] = train_set['girisimcilikle_ilgili_deneyiminiz_var_mi'].map({'evet': 1, 'hayır': 0})
train_set['ingilizce_biliyor_musunuz'] = train_set['ingilizce_biliyor_musunuz'].map({'evet': 1, 'hayır': 0})

/tmp/ipykernel_17/3621309780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['cinsiyet'] = train_set['cinsiyet'].map({'erkek': 1, 'kadın': 0})
/tmp/ipykernel_17/3621309780.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['universite_turu'] = train_set['universite_turu'].map({'özel': 1, 'devlet': 0})
/tmp/ipykernel_17/3621309780.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [180]:
# Kategorize etmek için mapping sözlüğü oluşturma
kategori_mapping = {
    'hazırlık': 0,      # Hazırlık sınıfı
    '1': 1,             # 1. sınıf
    '2': 2,             # 2. sınıf
    '3': 3,             # 3. sınıf
    '4': 4,             # 4. sınıf
    '5': 5,             # Mezun
    '6': 6,             # Yüksek lisans
    'mezun': 7,         # Mezun
}

# Mapping işlemi
train_set['universite_kacinci_sinif'] = train_set['universite_kacinci_sinif'].map(kategori_mapping)

/tmp/ipykernel_17/3743997769.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['universite_kacinci_sinif'] = train_set['universite_kacinci_sinif'].map(kategori_mapping)


In [181]:
# Kategorize etmek için mapping sözlüğü oluşturma
kategori_mapping = {
    '0': 0,     
    'ilkokul': 1,             
    'ortaokul': 2,            
    'lise': 3,            
    'universite': 4,            
    'yuksek lisans': 5,            
    'doktora': 6
}

# Mapping işlemi
train_set['baba_egitim_durumu'] = train_set['baba_egitim_durumu'].map(kategori_mapping)

/tmp/ipykernel_17/476770626.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['baba_egitim_durumu'] = train_set['baba_egitim_durumu'].map(kategori_mapping)


In [182]:
# Kategorize etmek için mapping sözlüğü oluşturma
kategori_mapping = {
    '2.50 ve üstü': 2,      
    '2.50 ve altı': 1,             
    '0': 0             }

# Mapping işlemi
train_set['universite_not_ortalamasi'] = train_set['universite_not_ortalamasi'].map(kategori_mapping)

/tmp/ipykernel_17/654149944.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['universite_not_ortalamasi'] = train_set['universite_not_ortalamasi'].map(kategori_mapping)


In [183]:
# Kategorilere ayırma (0-25, 25-50, 50-75, 75-100) ve numaralandırma (0, 1, 2, 3)
bins = [-1, 25, 50, 75, 100]
labels = [0, 1, 2, 3]  # Her aralığa karşılık gelen numaralar


# Verileri sınıflandırma
train_set['burslu_ise_burs_yuzdesi'] = pd.cut(train_set['burslu_ise_burs_yuzdesi'], bins=bins, labels=labels, right=True)

/tmp/ipykernel_17/2449349108.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['burslu_ise_burs_yuzdesi'] = pd.cut(train_set['burslu_ise_burs_yuzdesi'], bins=bins, labels=labels, right=True)


In [184]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None) # Satır sayısı görüntüleme.
pd.reset_option("display.max_rows")

In [185]:
# Pandas'ın get_dummies fonksiyonu ile dummy variable oluşturma.
#df_dummies = pd.get_dummies(train_set['cinsiyet'], drop_first=True)
#yeni = pd.concat([train_set, df_dummies], axis=1)  # Eski dataframe ile dummy variable'ları birleştir

In [186]:
train_set

,basvuru_yili,degerlendirme_puani,cinsiyet,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,universite_kacinci_sinif,universite_not_ortalamasi,daha_once_baska_bir_universiteden_mezun_olmus,baska_bir_kurumdan_burs_aliyor_mu,anne_calisma_durumu,anne_sektor,baba_egitim_durumu,baba_calisma_durumu,baba_sektor,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz,profesyonel_bir_spor_daliyla_mesgul_musunuz,aktif_olarak_bir_stk_uyesi_misiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,id
0,2014,52.0,1,1,3,1,3,2,0,0,0,0,4,2,0,1.0,1,1,0,1,0,0,0,0
1,2014,30.0,1,1,3,0,3,2,0,0,1,1,5,2,0,1.0,0,0,0,1,1,0,0,1
2,2014,18.0,1,1,3,0,1,0,0,0,0,0,1,1,2,6.0,1,0,0,0,0,0,0,2
3,2014,40.0,1,1,3,1,3,2,0,0,0,0,4,2,0,10.0,1,0,0,1,0,0,0,3
5,2014,37.0,1,1,3,1,3,2,0,0,0,0,5,1,1,3.0,1,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65119,2022,0.0,1,0,0,0,2,1,0,0,0,0,1,1,2,1.0,0,0,0,0,0,0,0,65119
65120,2022,0.0,1,1,0,1,2,1,0,0,0,0,2,1,2,4.0,0,0,0,0,0,0,0,65120
65121,2022,0.0,1,0,0,0,1,0,0,0,0,0,3,1,2,4.0,0,0,0,0,0,0,0,65121
65122,2022,0.0,1,0,0,0,4,1,0,1,0,0,3,0,0,3.0,0,0,0,0,0,0,0,65122


In [187]:
train_set.isna().sum()

basvuru_yili                                           0
degerlendirme_puani                                    0
cinsiyet                                               0
universite_turu                                        0
burslu_ise_burs_yuzdesi                                0
burs_aliyor_mu                                         0
universite_kacinci_sinif                               0
universite_not_ortalamasi                              0
daha_once_baska_bir_universiteden_mezun_olmus          0
baska_bir_kurumdan_burs_aliyor_mu                      0
anne_calisma_durumu                                    0
anne_sektor                                            0
baba_egitim_durumu                                     0
baba_calisma_durumu                                    0
baba_sektor                                            0
kardes_sayisi                                          0
girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz    0
profesyonel_bir_spor_daliyla_me

In [188]:
train_set['kardes_sayisi'] = train_set['kardes_sayisi'].astype('int64')
train_set['burslu_ise_burs_yuzdesi'] = train_set['burslu_ise_burs_yuzdesi'].astype('int64')

/tmp/ipykernel_17/2582981688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['kardes_sayisi'] = train_set['kardes_sayisi'].astype('int64')
/tmp/ipykernel_17/2582981688.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['burslu_ise_burs_yuzdesi'] = train_set['burslu_ise_burs_yuzdesi'].astype('int64')


# MODELLER

In [189]:
train_set.corr()

,basvuru_yili,degerlendirme_puani,cinsiyet,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,universite_kacinci_sinif,universite_not_ortalamasi,daha_once_baska_bir_universiteden_mezun_olmus,baska_bir_kurumdan_burs_aliyor_mu,anne_calisma_durumu,anne_sektor,baba_egitim_durumu,baba_calisma_durumu,baba_sektor,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz,profesyonel_bir_spor_daliyla_mesgul_musunuz,aktif_olarak_bir_stk_uyesi_misiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,id
basvuru_yili,1.000000,-0.145772,-0.130612,0.007290,-0.056918,-0.055901,-0.026479,0.202747,-0.118814,0.199505,-0.189641,0.032508,0.005529,-0.463981,0.078822,-0.080814,-0.098618,0.309614,0.099181,-0.411315,-0.149586,0.299986,-0.413204,0.759494
degerlendirme_puani,-0.145772,1.000000,-0.017320,0.128444,0.102847,0.096041,0.154849,0.145234,-0.005523,-0.018997,0.165408,0.079245,0.243107,0.125600,0.003584,-0.141835,0.592193,0.350226,0.453276,0.459472,0.362835,0.188715,0.221139,-0.172374
cinsiyet,-0.130612,-0.017320,1.000000,-0.024907,0.001796,-0.011958,-0.036638,-0.100672,0.024824,-0.041851,-0.022374,-0.059572,-0.004054,0.061399,-0.045063,0.045799,0.051548,0.055190,-0.018216,0.060432,0.162304,-0.034746,0.075231,-0.088750
universite_turu,0.007290,0.128444,-0.024907,1.000000,0.677488,0.790323,-0.040013,0.000306,-0.001897,-0.012843,0.115833,0.089548,0.165338,0.033781,0.061061,-0.131142,0.118232,0.049490,0.037155,0.049120,0.061612,0.080540,0.089161,-0.439905
burslu_ise_burs_yuzdesi,-0.056918,0.102847,0.001796,0.677488,1.000000,0.722551,-0.051216,0.017267,-0.008155,0.066590,0.065976,0.044232,0.080603,0.013558,0.008812,-0.066523,0.083819,0.015325,-0.020917,0.065996,0.034763,0.047672,0.109782,-0.574259
burs_aliyor_mu,-0.055901,0.096041,-0.011958,0.790323,0.722551,1.000000,-0.033929,0.003492,-0.003845,0.032544,0.080307,0.059388,0.090933,0.015955,0.027312,-0.052742,0.087731,0.017845,0.013731,0.045269,0.049832,0.035415,0.075803,-0.536721
universite_kacinci_sinif,-0.026479,0.154849,-0.036638,-0.040013,-0.051216,-0.033929,1.000000,0.307002,0.064056,-0.048454,0.036282,0.009963,0.045921,0.042426,-0.024924,-0.015014,0.061531,-0.016310,0.098910,0.064182,0.050773,-0.009583,-0.011283,0.019603
universite_not_ortalamasi,0.202747,0.145234,-0.100672,0.000306,0.017267,0.003492,0.307002,1.000000,-0.027544,0.075822,-0.045489,-0.006288,-0.017767,-0.133163,0.009382,0.001292,0.000225,0.098560,0.031328,-0.066904,-0.036308,0.038070,-0.130543,0.130602
daha_once_baska_bir_universiteden_mezun_olmus,-0.118814,-0.005523,0.024824,-0.001897,-0.008155,-0.003845,0.064056,-0.027544,1.000000,-0.046977,0.021730,-0.006659,-0.026112,0.073747,-0.027156,0.033318,-0.012073,-0.043476,-0.004859,0.051205,0.027401,-0.064301,0.020999,-0.086153
baska_bir_kurumdan_burs_aliyor_mu,0.199505,-0.018997,-0.041851,-0.012843,0.066590,0.032544,-0.048454,0.075822,-0.046977,1.000000,-0.086384,-0.044210,-0.132055,-0.145710,-0.041804,0.039300,-0.022169,0.038251,0.036717,-0.062299,-0.039370,0.045904,-0.057562,0.121467


In [190]:
import warnings

# Global olarak tüm uyarıları kapatmak için
warnings.filterwarnings("ignore")

# Kodunuz burada yer alacak

In [191]:
# Gerekli kütüphaneler
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Hedef ve özellik sütunları seçelim
X = train_set.drop("degerlendirme_puani", axis=1)  # Tahmin etmek istediğiniz sütun dışındaki veriler
y = train_set["degerlendirme_puani"]  # Hedef sütun

# Veriyi eğitim ve test setlerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [192]:
# ELASTIC NET
from sklearn.linear_model import ElasticNet
alpha = 0.7  # Modelin ceza gücü (daha yüksek alpha = daha fazla ceza)
l1_ratio = 0.5  # L1 (Lasso) ve L2 (Ridge) arasında denge (0 ile 1 arasında değer alır)

elastic_net_model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)

elastic_net_model.fit(X_train, y_train)

test_predictions = elastic_net_model.predict(X_test)

In [193]:
from sklearn.metrics import mean_squared_error, r2_score

# RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
print(f"RMSE: {rmse}")

# R² (R-squared)
r2 = r2_score(y_test, test_predictions)
print(f"R²: {r2}")

RMSE: 12.976687408557813
R²: 0.4982538160221598


# TEST SETİNİN DÜZENLENMESİ

In [194]:
test = pd.read_csv("/kaggle/input/datathon-2024/test_x.csv")

In [195]:
ids = test['id']

In [196]:
test['Cinsiyet'] = test['Cinsiyet'].replace('ERKEK', 'Erkek')
test["Cinsiyet"] = test["Cinsiyet"].fillna(0)
test["Cinsiyet"] = test["Cinsiyet"].replace('Belirtmek istemiyorum', '0')

# SÜTUN ADLARINI DÜZELTME
test.columns = [col.strip().lower().replace(' ', '_').replace('?', '').replace("'", '').replace("ö", 'o').replace("ü", 'u').replace("Ö", 'o').replace("Ü", 'u').replace(",", '') for col in test.columns]

test = test.applymap(lambda x: x.lower() if isinstance(x, str) else x)
test['cinsiyet'] = test['cinsiyet'].replace('belirtmek istemiyorum', '0')
test["universite_turu"] = test["universite_turu"].fillna(0)
test["cinsiyet"] = test["cinsiyet"].fillna(0)
test["burslu_ise_burs_yuzdesi"] = test["burslu_ise_burs_yuzdesi"].fillna(0)
#test = test.loc[test['universite_kacinci_sinif'] != '0']
#test = test.loc[test['universite_kacinci_sinif'] != 'tez']
#test = test.loc[test['universite_kacinci_sinif'] != 'yüksek lisans']
test["universite_kacinci_sinif"] = test["universite_kacinci_sinif"].fillna(0)
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('3.00-2.50', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('3.00 - 3.50', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('3.50-3', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('2.50 - 3.00', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('3.50 - 4.00', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('2.50 - 2.99', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('2.00 - 2.50', '2.50 ve altı')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('3.00 - 3.49', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('2.50 -3.00', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('3.00 - 4.00', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('1.80 - 2.49', '2.50 ve altı')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('4-3.5', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('1.00 - 2.50', '2.50 ve altı')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('0 - 1.79', '2.50 ve altı')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('4.0-3.5', '2.50 ve üstü')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('not ortalaması yok', '0')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('ortalama bulunmuyor', '0')
test['universite_not_ortalamasi'] = test['universite_not_ortalamasi'].replace('hazırlığım', '0')
test["universite_not_ortalamasi"] = test["universite_not_ortalamasi"].fillna("0")
test["daha_once_baska_bir_universiteden_mezun_olmus"] = test["daha_once_baska_bir_universiteden_mezun_olmus"].fillna("hayır")
test["lise_mezuniyet_notu"] = test["lise_mezuniyet_notu"].fillna("hayır")
test['baska_bir_kurumdan_burs_aliyor_mu'] = test['baska_bir_kurumdan_burs_aliyor_mu'].fillna(test['burs_aliyor_mu'])
test['anne_egitim_durumu'] = test['anne_egitim_durumu'].fillna("eğitim yok")
test['anne_calisma_durumu'] = test['anne_calisma_durumu'].fillna("hayır")
# 2 tane "diğer" algıladığı için, bunu düzeltme vakti
test["anne_sektor"] = test["anne_sektor"].str.lower()
test["anne_sektor"] = test["anne_sektor"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
test['anne_sektor'] = test['anne_sektor'].replace('-', '0')
test['anne_sektor'] = test['anne_sektor'].fillna("0")
test["baba_egitim_durumu"] = test["baba_egitim_durumu"].str.lower()
test["baba_egitim_durumu"] = test["baba_egitim_durumu"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
# "yüksek lisans / doktara"ları YÜKSEK LİSANSA YAZIYORUM, FARKETMEZ ŞİMDİLİK

test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('universite mezunu', 'universite')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('ilkokul mezunu', 'ilkokul')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('ortaokul mezunu', 'ortaokul')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('lise mezunu', 'lise')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('egitim yok', '0')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('egitimi yok', '0')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('yuksek lisans / doktora', 'yuksek lisans')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].replace('yuksek lisans / doktara', 'yuksek lisans')
test['baba_egitim_durumu'] = test['baba_egitim_durumu'].fillna("0")
# NaN verileri babası çalışmıyor'a yazdım. 

test['baba_calisma_durumu'] = test['baba_calisma_durumu'].fillna("hayır")

test["baba_sektor"] = test["baba_sektor"].str.lower()
test["baba_sektor"] = test["baba_sektor"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
test['baba_sektor'] = test['baba_sektor'].replace('-', '0')
# NaN verileri babası çalışmıyor'a yazdım. 

test['baba_sektor'] = test['baba_sektor'].fillna("0")
test["kardes_sayisi"] = pd.to_numeric(test["kardes_sayisi"], errors='coerce')
# NaN verileri babası çalışmıyor'a yazdım. 

test['kardes_sayisi'] = test['kardes_sayisi'].fillna("0")
# NaN verileri babası çalışmıyor'a yazdım. 

test['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'] = test['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'].fillna("hayır")

# NaN verileri babası çalışmıyor'a yazdım. 

test['profesyonel_bir_spor_daliyla_mesgul_musunuz'] = test['profesyonel_bir_spor_daliyla_mesgul_musunuz'].fillna("hayır")
test['aktif_olarak_bir_stk_uyesi_misiniz'] = test['aktif_olarak_bir_stk_uyesi_misiniz'].fillna("hayır")
test['stk_projesine_katildiniz_mi'] = test['stk_projesine_katildiniz_mi'].fillna("hayır")
test['girisimcilikle_ilgili_deneyiminiz_var_mi'] = test['girisimcilikle_ilgili_deneyiminiz_var_mi'].fillna("hayır")
test['ingilizce_biliyor_musunuz'] = test['ingilizce_biliyor_musunuz'].fillna("hayır")
#test["ingilizce_seviyeniz"] = test["ingilizce_seviyeniz"].str.lower()
#test["ingilizce_seviyeniz"] = test["ingilizce_seviyeniz"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
test['ingilizce_seviyeniz'] = test['ingilizce_seviyeniz'].fillna("0")

In [197]:
# 'selected_columns' listesinde tutmak istediğin sütunlar olacak.
# Normalde olmasına rağmen hızlanmam gerektiği için (submission zamanını kaçırmamak adına), lise_mezuniyet_notu ve anne_egitim_durumu'nu almıyorum.
selected_columns = ['basvuru_yili', 'cinsiyet',
       'universite_turu',
       'burslu_ise_burs_yuzdesi', 'burs_aliyor_mu',
       'universite_kacinci_sinif', 'universite_not_ortalamasi',
       'daha_once_baska_bir_universiteden_mezun_olmus',
       'baska_bir_kurumdan_burs_aliyor_mu',
       'anne_calisma_durumu', 'anne_sektor', 'baba_egitim_durumu',
       'baba_calisma_durumu', 'baba_sektor', 'kardes_sayisi',
       'girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz',
       'profesyonel_bir_spor_daliyla_mesgul_musunuz','aktif_olarak_bir_stk_uyesi_misiniz',
       'stk_projesine_katildiniz_mi',
       'girisimcilikle_ilgili_deneyiminiz_var_mi',
       'ingilizce_biliyor_musunuz', 'ingilizce_seviyeniz',
       'id']  # Bu sütunları sakla

test_set = test[selected_columns]  # Geri kalan sütunlar drop edilmiş olur.


test_set['cinsiyet'] = test_set['cinsiyet'].map({'erkek': 1, 'kadın': 0})
test_set['universite_turu'] = test_set['universite_turu'].map({'özel': 1, 'devlet': 0})
test_set['burs_aliyor_mu'] = test_set['burs_aliyor_mu'].map({'evet': 1, 'hayır': 0})
test_set['daha_once_baska_bir_universiteden_mezun_olmus'] = test_set['daha_once_baska_bir_universiteden_mezun_olmus'].map({'evet': 1, 'hayır': 0})
test_set['baska_bir_kurumdan_burs_aliyor_mu'] = test_set['baska_bir_kurumdan_burs_aliyor_mu'].map({'evet': 1, 'hayır': 0})
test_set['anne_calisma_durumu'] = test_set['anne_calisma_durumu'].map({'evet': 1, 'hayır': 0, 'emekli':2})
test_set['anne_sektor'] = test_set['anne_sektor'].map({'0': 0, 'kamu': 1, 'ozel sektor':2, 'diger':3})
test_set['baba_calisma_durumu'] = test_set['baba_calisma_durumu'].map({'evet': 1, 'hayır': 0, 'emekli':2})
test_set['baba_sektor'] = test_set['baba_sektor'].map({'0': 0, 'kamu': 1, 'ozel sektor':2, 'diger':3})
test_set['ingilizce_seviyeniz'] = test_set['ingilizce_seviyeniz'].map({'0': 0, 'baslangc': 1, 'orta':2, 'ileri':3})
test_set['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'] = test_set['girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz'].map({'evet': 1, 'hayır': 0})
test_set['profesyonel_bir_spor_daliyla_mesgul_musunuz'] = test_set['profesyonel_bir_spor_daliyla_mesgul_musunuz'].map({'evet': 1, 'hayır': 0})
test_set['aktif_olarak_bir_stk_uyesi_misiniz'] = test_set['aktif_olarak_bir_stk_uyesi_misiniz'].map({'evet': 1, 'hayır': 0})
test_set['stk_projesine_katildiniz_mi'] = test_set['stk_projesine_katildiniz_mi'].map({'evet': 1, 'hayır': 0})
test_set['girisimcilikle_ilgili_deneyiminiz_var_mi'] = test_set['girisimcilikle_ilgili_deneyiminiz_var_mi'].map({'evet': 1, 'hayır': 0})
test_set['ingilizce_biliyor_musunuz'] = test_set['ingilizce_biliyor_musunuz'].map({'evet': 1, 'hayır': 0})


# Kategorize etmek için mapping sözlüğü oluşturma
kategori_mapping = {
    'hazırlık': 0,      # Hazırlık sınıfı
    '1': 1,             # 1. sınıf
    '2': 2,             # 2. sınıf
    '3': 3,             # 3. sınıf
    '4': 4,             # 4. sınıf
    '5': 5,             # Mezun
    '6': 6,             # Yüksek lisans
    'mezun': 7,         # Mezun
}

# Mapping işlemi
test_set['universite_kacinci_sinif'] = test_set['universite_kacinci_sinif'].map(kategori_mapping)


# Kategorize etmek için mapping sözlüğü oluşturma
kategori_mapping = {
    '0': 0,     
    'ilkokul': 1,             
    'ortaokul': 2,            
    'lise': 3,            
    'universite': 4,            
    'yuksek lisans': 5,            
    'doktora': 6
}

# Mapping işlemi
test_set['baba_egitim_durumu'] = test_set['baba_egitim_durumu'].map(kategori_mapping)


# Kategorize etmek için mapping sözlüğü oluşturma
kategori_mapping = {
    '2.50 ve üstü': 2,      
    '2.50 ve altı': 1,             
    '0': 0             }

# Mapping işlemi
test_set['universite_not_ortalamasi'] = test_set['universite_not_ortalamasi'].map(kategori_mapping)

# Kategorilere ayırma (0-25, 25-50, 50-75, 75-100) ve numaralandırma (0, 1, 2, 3)
bins = [-1, 25, 50, 75, 100]
labels = [0, 1, 2, 3]  # Her aralığa karşılık gelen numaralar


# Verileri sınıflandırma
test_set['burslu_ise_burs_yuzdesi'] = pd.cut(test_set['burslu_ise_burs_yuzdesi'], bins=bins, labels=labels, right=True)

test_set['kardes_sayisi'] = test_set['kardes_sayisi'].astype('int64')
test_set['burslu_ise_burs_yuzdesi'] = test_set['burslu_ise_burs_yuzdesi'].astype('int64')


test_set["cinsiyet"] = test_set["cinsiyet"].fillna(0)

In [198]:
test_set.isna().sum()

basvuru_yili                                           0
cinsiyet                                               0
universite_turu                                        0
burslu_ise_burs_yuzdesi                                0
burs_aliyor_mu                                         0
universite_kacinci_sinif                               0
universite_not_ortalamasi                              0
daha_once_baska_bir_universiteden_mezun_olmus          0
baska_bir_kurumdan_burs_aliyor_mu                      0
anne_calisma_durumu                                    0
anne_sektor                                            0
baba_egitim_durumu                                     0
baba_calisma_durumu                                    0
baba_sektor                                            0
kardes_sayisi                                          0
girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz    0
profesyonel_bir_spor_daliyla_mesgul_musunuz            0
aktif_olarak_bir_stk_uyesi_misi

In [199]:
test_set

,basvuru_yili,cinsiyet,universite_turu,burslu_ise_burs_yuzdesi,burs_aliyor_mu,universite_kacinci_sinif,universite_not_ortalamasi,daha_once_baska_bir_universiteden_mezun_olmus,baska_bir_kurumdan_burs_aliyor_mu,anne_calisma_durumu,anne_sektor,baba_egitim_durumu,baba_calisma_durumu,baba_sektor,kardes_sayisi,girisimcilik_kulupleri_tarzi_bir_kulube_uye_misiniz,profesyonel_bir_spor_daliyla_mesgul_musunuz,aktif_olarak_bir_stk_uyesi_misiniz,stk_projesine_katildiniz_mi,girisimcilikle_ilgili_deneyiminiz_var_mi,ingilizce_biliyor_musunuz,ingilizce_seviyeniz,id
0,2023,0.0,0,0,0,1,2,0,0,1,2,2,1,3,2,0,1,1,0,0,1,0,0
1,2023,0.0,0,0,0,1,0,0,0,0,0,1,1,2,3,0,1,0,0,0,1,0,1
2,2023,1.0,1,0,1,1,0,0,1,0,0,4,0,0,0,0,0,0,0,0,1,0,2
3,2023,1.0,0,0,0,1,0,0,0,0,0,3,1,2,4,0,0,1,0,0,0,0,3
4,2023,1.0,0,0,0,3,2,0,1,1,1,4,1,1,1,0,0,1,0,1,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,0.0,0,0,0,2,1,0,1,0,0,2,0,0,0,1,0,1,0,0,1,0,11044
11045,2023,0.0,0,0,0,3,2,0,0,0,0,1,1,1,3,1,0,1,0,0,0,0,11045
11046,2023,1.0,0,0,0,1,0,0,0,0,0,4,1,3,2,1,1,1,0,0,1,0,11046
11047,2023,0.0,0,0,0,4,1,0,0,0,0,2,1,3,2,1,1,1,0,1,1,0,11047


# TEST MODELİ VE SUBMISSION

In [200]:
sub = pd.read_csv("/kaggle/input/datathon-2024/sample_submission.csv")

In [201]:
predictions = elastic_net_model.predict(test_set)

In [202]:
predictions

array([34.13981831, 23.85106515, 25.04668644, ..., 38.00118046,
       43.02146685, 33.47578275])

In [203]:
sub

,id,Degerlendirme Puani
0,0,1.0
1,1,27.0
2,2,17.0


In [204]:
print("test id length:", len(ids))
print("predictions length:", len(predictions))

test id length: 11049
predictions length: 11049


In [205]:
submission = pd.DataFrame({
    'id': ids,  # 'id' sütununu test dosyasından alın
    'Degerlendirme Puani': predictions  # Tahminleri 'Degerlendirme Puani'na aktar
})

In [206]:
submission

,id,Degerlendirme Puani
0,0,34.139818
1,1,23.851065
2,2,25.046686
3,3,24.557027
4,4,37.188058
...,...,...
11044,11044,35.148911
11045,11045,33.680581
11046,11046,38.001180
11047,11047,43.021467


In [207]:
submission.to_csv('submission.csv', index=False)

In [208]:
output = pd.read_csv("/kaggle/working/submission.csv")

In [209]:
output

,id,Degerlendirme Puani
0,0,34.139818
1,1,23.851065
2,2,25.046686
3,3,24.557027
4,4,37.188058
...,...,...
11044,11044,35.148911
11045,11045,33.680581
11046,11046,38.001180
11047,11047,43.021467
